# step 1

In [2]:
import pandas as pd
import numpy as np

file = 'E://ggl/sqlResult_1558435.csv'

content = pd.read_csv(file, encoding='gb18030')

In [20]:
content.head(5)

,id,author,source,content,feature,title,url,xinhua
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm,0.0
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm,0.0
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm,0.0
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....,1.0
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...,0.0


# step 2

In [4]:
content['xinhua'] = np.nan

In [5]:
for i in range(len(content)):
    if content.iloc[i, 2] == '新华社':
        content.loc[i, 'xinhua'] = 1
    else:
        content.loc[i, 'xinhua'] = 0

In [6]:
X, y = content[['content']], content[['xinhua']]

In [19]:
X[:5]

       xinhua
0         0.0
1         0.0
2         0.0
3         1.0
4         0.0
5         0.0
6         0.0
7         0.0
8         0.0
9         0.0
10        0.0
11        0.0
12        0.0
13        0.0
14        0.0
15        0.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        0.0
24        0.0
25        0.0
26        0.0
27        0.0
28        0.0
29        0.0
...       ...
89581     1.0
89582     1.0
89583     1.0
89584     1.0
89585     1.0
89586     1.0
89587     1.0
89588     1.0
89589     1.0
89590     1.0
89591     1.0
89592     1.0
89593     1.0
89594     1.0
89595     1.0
89596     1.0
89597     1.0
89598     1.0
89599     1.0
89600     1.0
89601     1.0
89602     1.0
89603     1.0
89604     1.0
89605     1.0
89606     1.0
89607     1.0
89608     1.0
89609     1.0
89610     1.0

[89611 rows x 1 columns]


In [8]:
import jieba
import re

In [9]:
new_X = []
for i in range(len(X)):
    tmp = re.sub('[\\a-zA-Z0-9，。（）/：…@！？\s\n]', '', str(X.iloc[i, 0]))
    new_X.append(' '.join(jieba.cut(tmp)))
new_X = pd.Series(new_X)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\guoli\AppData\Local\Temp\jieba.cache
Loading model cost 0.764 seconds.
Prefix dict has been built succesfully.


In [10]:
new_X[0]

'此外 自 本周 月 日 起 除 小米 手机 等款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 的 研发 之中 去年 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 的 确切 信息 我们 还是 等待 官方消息'

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# step 3

In [12]:
transformer = TfidfVectorizer()
X_tfidf = transformer.fit_transform(new_X)
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [13]:
X_train_validation, X_test, y_train_validation, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=1988)
X_train, X_validation, y_train, y_validation = train_test_split(X_train_validation, y_train_validation, test_size=0.25, random_state=1988)

print(X_train))

<class 'scipy.sparse.csr.csr_matrix'>


# step 4

# create models

## model 1: logistic regression

In [15]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression()
model_lg = lg.fit(X_train, y_train)
pred_lg = lg.predict(X_validation)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
from sklearn.metrics import confusion_matrix

# step 5

In [27]:
confusion = confusion_matrix(y_validation, pred_lg)

In [28]:
def evoluate(num_py):
    acc_lr = (num_py[0][0]+num_py[1][1]) / sum([e for e in sum(num_py)]) * 100
    precision_lr = num_py[0][0] / (num_py[0][0]+num_py[0][1]) * 100
    recall_lr = num_py[0][0] / (num_py[0][0]+num_py[1][0]) * 100
    f1_lr = 2 * precision_lr * recall_lr / (precision_lr+recall_lr)
    print('acceracy: ', f'{acc_lr}', '%; ', 'precision: ', f'{precision_lr}', '%;\n',
          'recall: ', f'{recall_lr}', '%.', 'f1 score: ', f'{f1_lr}', sep='')


In [29]:
evoluate(confusion)

acceracy: 96.76375404530745%; precision: 78.15013404825737%;
recall: 95.05434782608695%.f1 score: 85.77734183423247


![](Precisionrecall.svg.png)

- Accuracy is the fraction of correct prediction.
- Precision is the the fraction of relevant instances among the retrieved instances.
- Recall is the fraction of relevant instances that have been retrieved over the total amount of relevant instances.
- F1 score is the combination of precision and recall.
<div align="right">-- https://en.wikipedia.org/wiki/Precision_and_recall</div>

## model 2: decision tree

In [30]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
model_dt = dt.fit(X_train, y_train)
pred_dt = model_dt.predict(X_validation)
tree_confusion= confusion_matrix(y_validation, pred_dt)


NameError: name 'tree_confusiony' is not defined

In [31]:
evoluate(tree_confusion)

acceracy: 98.93427072871332%; precision: 96.55942806076855%;
recall: 94.98901098901099%.f1 score: 95.76778196321737


## model 3: random forest

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
model_rf = rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_validation)
evoluate(confusion_matrix(y_validation, pred_rf))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


acceracy: 95.68686530521148%; precision: 70.10723860589813%;
recall: 93.78362223550508%.f1 score: 80.23523395551011


### The false negative is much more than decision tree.

## model 4: SVM

In [33]:
from sklearn.svm import SVC
svm = SVC()
model_svm = svm.fit(X_train, y_train)
pred_svm = model_svm.predict(X_validation)
evoluate(confusion_matrix(y_validation, pred_svm))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acceracy: 87.51255440241044%; precision: 0.0%;
recall: nan%.f1 score: nan


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in longlong_scalars
  after removing the cwd from sys.path.


### This model (SVM) performs badly. I won't calculate model evaluation.

***
## Decision Tree outperforms other model. Try to optimize it.

In [22]:
model_dt

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

### Change criterion:

In [23]:
dt2 = DecisionTreeClassifier(criterion='entropy')
model_dt2 = dt2.fit(X_train, y_train)
pred_dt2 = model_dt2.predict(X_validation)
confusion_matrix(y_validation, pred_dt2)
# a slightly worse than original model

array([[ 2142,    96],
       [  113, 15571]])

In [38]:
acc_dt2 = (2142+15584) / (2166+96+113+15571)
precision_dt2 = 2142 / (2162+96)
recall_dt2 = 2142 / (2142+113)
f1_dt2 = 2 * precision_dt2 * recall_dt2 / (precision_dt2+recall_dt2)
print('acceracy: ', f'{acc_dt2}', '%; ', 'precision: ', f'{precision_dt2}', '%;\n',
      'recall: ', f'{recall_dt2}', '%.', 'f1 score: ', f'{f1_dt2}', sep='')

acceracy: 0.9877410007801182%; precision: 0.9486271036315324%;
recall: 0.9498891352549889%.f1 score: 0.9492576999778418


### Tune max_dapth:

In [25]:
dt3 = DecisionTreeClassifier(criterion='gini', max_depth=10)
model_dt3 = dt3.fit(X_train, y_train)
pred_dt3 = model_dt3.predict(X_validation)
confusion_matrix(y_validation, pred_dt3)

array([[ 2167,    71],
       [  105, 15579]])

In [39]:
acc_dt3 = (2167+15579) / (2167+71+105+15579)
precision_dt3 = 2167 / (2167+66)
recall_dt3 = 2167 / (2167+105)
f1_dt3 = 2 * precision_dt3 * recall_dt3 / (precision_dt3+recall_dt3)
print('acceracy: ', f'{acc_dt3}', '%; ', 'precision: ', f'{precision_dt3}', '%;\n',
      'recall: ', f'{recall_dt3}', '%.', 'f1 score: ', f'{f1_dt3}', sep='')

acceracy: 0.9901796674478295%; precision: 0.9704433497536946%;
recall: 0.9537852112676056%.f1 score: 0.9620421753607102


### The third model performs best. Tune min_samples_leaf based on it.

In [27]:
dt4 = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_leaf=2)
model_dt4 = dt4.fit(X_train, y_train)
pred_dt4 = model_dt4.predict(X_validation)
confusion_matrix(y_validation, pred_dt4)

array([[ 2167,    71],
       [  108, 15576]])

In [40]:
acc_dt3 = (2167+15576) / (2167+71+108+15576)
precision_dt3 = 2167 / (2167+71)
recall_dt3 = 2167 / (2167+108)
f1_dt3 = 2 * precision_dt3 * recall_dt3 / (precision_dt3+recall_dt3)
print('acceracy: ', f'{acc_dt3}', '%; ', 'precision: ', f'{precision_dt3}', '%;\n',
      'recall: ', f'{recall_dt3}', '%.', 'f1 score: ', f'{f1_dt3}', sep='')

acceracy: 0.9900122754156903%; precision: 0.9682752457551386%;
recall: 0.9525274725274725%.f1 score: 0.9603368047861732


### The third model still performs best. Next tune min_samples_split.

In [29]:
dt5 = DecisionTreeClassifier(criterion='gini', max_depth=10, min_samples_split=3)
model_dt5 = dt5.fit(X_train, y_train)
pred_dt5 = model_dt5.predict(X_validation)
confusion_matrix(y_validation, pred_dt5)

array([[ 2168,    70],
       [  105, 15579]])

In [41]:
acc_dt5 = (2168+15581) / (2168+70+105+15579)
precision_dt5 = 2168 / (2168+70)
recall_dt5 = 2168 / (2168+105)
f1_dt5 = 2 * precision_dt5 * recall_dt5 / (precision_dt5+recall_dt5)
print('acceracy: ', f'{acc_dt5}', '%; ', 'precision: ', f'{precision_dt5}', '%;\n',
      'recall: ', f'{recall_dt5}', '%.', 'f1 score: ', f'{f1_dt5}', sep='')

acceracy: 0.9903470594799687%; precision: 0.968722073279714%;
recall: 0.9538055433347998%.f1 score: 0.9612059410330304


### Still the third model preforms best. Last to tune min_weight_fraction_leaf.

In [34]:
dt6 = DecisionTreeClassifier(criterion='gini', max_depth=10, min_weight_fraction_leaf=0.01)
model_dt6 = dt6.fit(X_train, y_train)
pred_dt6 = model_dt6.predict(X_validation)
evoluate(confusion_matrix(y_validation, pred_dt6))

acceracy: 97.63419261243165%; precision: 91.24218051831993%;
recall: 89.95594713656388%.f1 score: 90.59449866903283


### The accuracy significantly decrease. This model won't be used.

# The conclusion is model 3 outperforms other models. Will use it to detect plagiarism.

***
## split the dataset without TF-IDF

In [32]:
# only X_test_contest is useful, I don't care others.
X_train_validation_contest, X_test_contest, y_train_validation, y_test = train_test_split(new_X, y, test_size=0.2, random_state=1988)

In [33]:
model_dt_best = dt3.fit(X_train_validation, y_train_validation) # use a larger dataset
predict = model_dt_best.predict(X_test)

In [34]:
predict = pd.Series(predict, index=X_test_contest.index)
articles = pd.DataFrame({'articles':X_test_contest, 'citation':y_test, 'predict':predict})

In [35]:
plagiarism = articles.loc[(articles['citation']==0) & (articles['predict']==1)]
# https://stackoverflow.com/questions/13611065/efficient-way-to-apply-multiple-filters-to-pandas-dataframe-or-series

In [36]:
len(plagiarism)

0

# The conclusion is that there is no plagiarism!